In [1]:
from datetime import date, timedelta
from collections import deque

### Is_consistent

In [2]:
# Function to check if a given assignment is consistent
def is_consistent(assignment, var, value):
    cur_date, cur_shift, cur_role = var.split("_")
    cur_date = date.fromisoformat(cur_date)
    
    # Hard constraints
    for assigned_var, assigned_value in assignment.items():
        assigned_date, assigned_shift, assigned_role = assigned_var.split("_")
        assigned_date = date.fromisoformat(assigned_date)

        # No person should be assigned two different roles in the same shift
        if cur_date == assigned_date and cur_shift == assigned_shift and value == assigned_value:
            return False

        # Morning-only and evening-only staff constraints
        if value in morning_only and cur_shift == 'Evening':
            return False
        if value in evening_only and cur_shift == 'Morning':
            return False
        
        # Separate work schedule for ASG
        if cur_role == "ASG" and assigned_role == "ASG":
            if cur_date == assigned_date and cur_shift == assigned_shift:
                return False
    
    
    return True

### AC3

In [3]:
# AC-3 algorithm to reduce domains
def ac3(domains):
    # Initialize queue with all arcs
    queue = deque()
    for x in domains:
        for y in domains:
            if x != y:
                queue.append((x, y))

    while queue:
        x, y = queue.popleft()
        if remove_inconsistent_values(domains, x, y):
            for z in domains:
                if z != x and z != y:
                    queue.append((z, x))

    return True

### Remove inconsistent values

In [4]:
# Helper function for AC-3
def remove_inconsistent_values(domains, x, y):
    removed = False
    for x_val in domains[x][:]:
        assignment = {x: x_val}
        if all(not is_consistent(assignment, y, y_val) for y_val in domains[y]):
            domains[x].remove(x_val)
            removed = True
    return removed

### # Additional function to check soft constraints

In [5]:
def check_soft_constraints(assignment, history, var, value):
    cur_date, cur_shift, cur_role = var.split("_")
    cur_date = date.fromisoformat(cur_date)
    
    # Soft Constraint: If possible, whoever has Sunday off has Monday off as well
    if cur_date.weekday() == 0:  # Monday
        prev_sunday = cur_date - timedelta(days=1)
        prev_var = f"{prev_sunday}_{cur_shift}_{cur_role}"
        if prev_var not in assignment or assignment[prev_var] != value:
            return False
    return True

### BackTracking

In [6]:
def backtrack(assignment, domains, history):
    if len(assignment) == len(domains):
        return assignment
    
    # Select an unassigned variable
    for var in domains.keys():
        if var not in assignment:
            break
    
    for value in domains[var]:
        if is_consistent(assignment, var, value) and check_soft_constraints(assignment, history, var, value):
            assignment[var] = value
            history[value].append(var)
            
            # If hard constraints are met, continue
            result = backtrack(assignment, domains, history)
            if result:
                return result
            
            # Revert changes
            del assignment[var]
            history[value].remove(var)
    
    return None

In [7]:
def backtrack_without_soft(assignment, domains, history):
    if len(assignment) == len(domains):
        return assignment
    
    # Select an unassigned variable
    for var in domains.keys():
        if var not in assignment:
            break
    
    for value in domains[var]:
        if is_consistent(assignment, var, value):
            assignment[var] = value
            history[value].append(var)
            
            # If hard constraints are met, continue
            result = backtrack(assignment, domains, history)
            if result:
                return result
            
            # Revert changes
            del assignment[var]
            history[value].remove(var)
    
    return None

###  Define the roles, shifts, and dates

In [8]:
# Define the roles, shifts, and dates
roles = ['CHEF', 'COZ', 'ASG', 'AUX', 'CONF', 'PIZ', 'GARD', 'MASSA']
shifts = ['Morning', 'Evening']
dates = [date(2023, 9, day) for day in range(1, 31)]

### Define constraints for employees who only work morning or evening shifts

In [9]:
morning_only = ['CHEF1', 'COZ1', 'COZ2', 'ASG1', 'ASG2', 'AUX1', 'CONF1', 'PIZ1', 'GARD1', 'MASSA1']
evening_only = ['CHEF2', 'COZ3', 'COZ4', 'ASG3', 'ASG4', 'AUX2', 'AUXPIZ1', 'PIZ2', 'GARD2', 'MASSA2']

### Initialize domains for each variable

In [10]:
# Initialize domains for each variable
domains = {}
for current_date in dates:
    for shift in shifts:
        for role in roles:
            var = f"{current_date}_{shift}_{role}"
            
            # Special rule for Sunday Evening Shifts
            if current_date.weekday() == 6 and shift == 'Evening':
                if role == 'CHEF':
                    domains[var] = ['CHEF1', 'CHEF2']
                elif role == 'PIZ':
                    domains[var] = ['PIZ1', 'PIZ2', 'AUXPIZ1']
                else:
                    domains[var] = []
                continue
            
            # Role-specific initialization based on the shift
            if shift == 'Morning':
                if role == 'CHEF':
                    domains[var] = ['CHEF1']
                elif role == 'COZ':
                    domains[var] = ['COZ1', 'COZ2']
                elif role == 'ASG':
                    domains[var] = ['ASG1', 'ASG2']
                elif role == 'AUX':
                    domains[var] = ['AUX1', 'CONF1', 'AUXPIZ1']
                elif role == 'CONF':
                    domains[var] = ['CONF1', 'AUXPIZ1']
                elif role == 'PIZ':
                    domains[var] = ['PIZ1', 'AUXPIZ1']
                elif role == 'GARD':
                    domains[var] = ['GARD1']
                elif role == 'MASSA':
                    domains[var] = ['MASSA1']
            else:  # Evening
                if role == 'CHEF':
                    domains[var] = ['CHEF2']
                elif role == 'COZ':
                    domains[var] = ['COZ3', 'COZ4']
                elif role == 'ASG':
                    domains[var] = ['ASG3', 'ASG4']
                elif role == 'AUX':
                    domains[var] = ['AUX2', 'CONF1', 'AUXPIZ1']
                elif role == 'CONF':
                    domains[var] = ['CONF1', 'AUXPIZ1']
                elif role == 'PIZ':
                    domains[var] = ['PIZ2', 'AUXPIZ1']
                elif role == 'GARD':
                    domains[var] = ['GARD2']
                elif role == 'MASSA':
                    domains[var] = ['MASSA2']

In [11]:
# Initialize history for each employee
history = {}
all_employees = morning_only + evening_only
for emp in all_employees:
    history[emp] = []

### Main program

In [12]:
def main():
    # Apply AC-3 to reduce domains
    if not ac3(domains):
        print("No solution exists after AC-3!")
        exit(1)

    # Initialize history for each employee
    history = {}
    all_employees = morning_only + evening_only
    for emp in all_employees:
        history[emp] = []
    
    # First attempt: Try to find a solution satisfying all constraints
    assignment = {}
    result = backtrack(assignment, domains, history)
    if result:
        print("Solution found with soft constraints:")
        for key, value in sorted(result.items()):
            print(f"{key}: {value}")
        return

    print("No solution with soft constraints. Attempting without...")
    
    assignment = {}
    result = backtrack_without_soft(assignment, domains, history)
    if result:
        print("Solution found without soft constraints:")
        for key, value in sorted(result.items()):
            print(f"{key}: {value}")
    else:
        print("No solution exists!")

if __name__ == "__main__":
    main()

No solution with soft constraints. Attempting without...
No solution exists!
